In [12]:
import threading

# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
        
class StreamingThread(threading.Thread):
    def __init__(self, ssc):
        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [13]:
sc

<SparkContext master=local[2] appName=PySparkShell>

In [14]:
spark

In [15]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, length
from pyspark.ml import PipelineModel
from pyspark.streaming import StreamingContext
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
from pyspark.sql.types import StructType, StructField, StringType, BooleanType, LongType, DoubleType
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier, GBTClassificationModel
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [16]:
# Initialize the SparkSession
spark = SparkSession.builder \
    .appName("FrontpagePrediction") \
    .config("spark.executor.memory", "2g") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.cores", "2") \
    .config("spark.driver.cores", "2") \
    .getOrCreate()


# Define the data schema from the json
schema = StructType([
    StructField("aid", StringType(), True),
    StructField("title", StringType(), True),
    StructField("url", StringType(), True),
    StructField("domain", StringType(), True),
    StructField("votes", LongType(), True),
    StructField("user", StringType(), True),
    StructField("posted_at", StringType(), True),
    StructField("comments", LongType(), True),
    StructField("source_title", StringType(), True),
    StructField("source_text", StringType(), True),
    StructField("frontpage", BooleanType(), True)
])

# Define a function to convert correct probability
def get_prediction(probability):
    return 0.0 if probability[1] <= 0.5 else 1.0

get_prediction_udf = udf(get_prediction, DoubleType())

# Define main process function
def process(time, rdd):
    if rdd.isEmpty():
        return
    
    print("========= %s =========" % str(time))
    
    # Convert rdd into data frame with specific schema
    df = spark.read.schema(schema).json(rdd)
    df.show(n=20)  # display the data we are capturing
    
    # Convert frontpage from boolean to string
    df = df.withColumn("frontpage", col("frontpage").cast("string"))
    # Add a new feature 'text_length'
    df = df.withColumn("text_length", length(col("source_text")))
    
    # Load the saved pipeline
    pipeline_model = PipelineModel.load("/Users/xiaodi/anaconda3/spark/notebooks/saved_pipeline")
    
    # Employ pipeline to convert and extract features
    df_transformed = pipeline_model.transform(df)
    
    # Load the saved GBT model
    if not globals().get('models_loaded', False):
        globals()['my_model'] = GBTClassificationModel.load("/Users/xiaodi/anaconda3/spark/notebooks/saved_gbt_model")
        globals()['models_loaded'] = True
    
    # Make predictions
    df_result = globals()['my_model'].transform(df_transformed)
    
    # Convert result into correct prediction
    df_result = df_result.withColumn("corrected_prediction", get_prediction_udf(col("probability")))
    
    # Display the first 20 lines
    df_result.show(n=20)
    
    
    # Save the results to csv file
    df_result.select("aid", "title", "url", "domain", "votes", "user", "posted_at", 
                     "comments", "frontpage", 
                     "corrected_prediction") \
             .write.mode("append").option("header", "true").csv("/Users/xiaodi/anaconda3/spark/notebooks/predictions")




In [17]:
ssc = StreamingContext(sc, 10)

In [18]:
lines = ssc.socketTextStream("seppe.net", 7778)
lines.foreachRDD(process)

In [19]:
ssc_t = StreamingThread(ssc)
ssc_t.start()

24/05/25 20:48:40 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/25 20:48:40 WARN BlockManager: Block input-0-1716662920000 replicated to only 0 peer(s) instead of 1 peers
24/05/25 20:48:41 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/25 20:48:41 WARN BlockManager: Block input-0-1716662921000 replicated to only 0 peer(s) instead of 1 peers
24/05/25 20:48:42 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/25 20:48:42 WARN BlockManager: Block input-0-1716662922000 replicated to only 0 peer(s) instead of 1 peers
24/05/25 20:48:43 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/25 20:48:43 WARN BlockManager: Block input-0-1716662923000 replicated to only 0 peer(s) instead of 1 peers
24/05/25 20:48:46 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/25 20:48:46 WARN BlockManager: Block input-0-1716662926000 replicated to

========= 2024-05-25 20:48:50 =========
+--------+--------------------+--------------------+--------------------+-----+--------------+-------------------+--------+--------------------+--------------------+---------+
|     aid|               title|                 url|              domain|votes|          user|          posted_at|comments|        source_title|         source_text|frontpage|
+--------+--------------------+--------------------+--------------------+-----+--------------+-------------------+--------+--------------------+--------------------+---------+
|40473125|Where can I find ...|https://github.co...|github.com/turbob...|    2|    Turboblack|2024-05-25 06:50:32|       2|GitHub - turbobla...|GitHub - turbobla...|    false|
|40473128|  EU Approves AI Act|https://vcsi.org/...|            vcsi.org|   25|  rippeltippel|2024-05-25 06:50:54|      24|EU Approves Histo...|EU Approves Histo...|     true|
|40473130|Signal's Meredith...|https://techcrunc...|      techcrunch.com|   10| 

24/05/25 20:48:51 WARN DAGScheduler: Broadcasting large task binary with size 1418.6 KiB
24/05/25 20:48:51 WARN DAGScheduler: Broadcasting large task binary with size 1418.6 KiB
24/05/25 20:48:51 WARN DAGScheduler: Broadcasting large task binary with size 1418.6 KiB
24/05/25 20:48:52 WARN DAGScheduler: Broadcasting large task binary with size 1414.7 KiB


+--------+--------------------+--------------------+--------------------+-----+--------------+-------------------+--------+--------------------+--------------------+---------+-----------+--------------------+--------------------+--------------------+--------------------+------------+-----------------+--------------------+-----+--------------------+--------------------+----------+--------------------+
|     aid|               title|                 url|              domain|votes|          user|          posted_at|comments|        source_title|         source_text|frontpage|text_length|               words|      filtered_words|         tf_features|      tfidf_features|domain_index|       domain_vec|            features|label|       rawPrediction|         probability|prediction|corrected_prediction|
+--------+--------------------+--------------------+--------------------+-----+--------------+-------------------+--------+--------------------+--------------------+---------+-----------+-----

24/05/25 20:48:52 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/25 20:48:52 WARN BlockManager: Block input-0-1716662932000 replicated to only 0 peer(s) instead of 1 peers
24/05/25 20:48:54 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/25 20:48:54 WARN BlockManager: Block input-0-1716662934200 replicated to only 0 peer(s) instead of 1 peers
24/05/25 20:48:57 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/25 20:48:57 WARN BlockManager: Block input-0-1716662937200 replicated to only 0 peer(s) instead of 1 peers
24/05/25 20:48:59 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/25 20:48:59 WARN BlockManager: Block input-0-1716662939200 replicated to only 0 peer(s) instead of 1 peers


...

24/05/25 20:59:21 WARN DAGScheduler: Broadcasting large task binary with size 1414.7 KiB
24/05/25 21:01:16 WARN RandomBlockReplicationPolicy: Expecting 1 replicas with only 0 peer/s.
24/05/25 21:01:16 WARN BlockManager: Block input-0-1716663676600 replicated to only 0 peer(s) instead of 1 peers


========= 2024-05-25 21:01:20 =========
+--------+--------------------+--------------------+-------------+-----+--------+-------------------+--------+--------------------+--------------------+---------+
|     aid|               title|                 url|       domain|votes|    user|          posted_at|comments|        source_title|         source_text|frontpage|
+--------+--------------------+--------------------+-------------+-----+--------+-------------------+--------+--------------------+--------------------+---------+
|40474779|Production of nex...|https://www.autoc...|autocar.co.uk|    1|peutetre|2024-05-25 13:01:03|       0|Production of nex...|Production of nex...|    false|
+--------+--------------------+--------------------+-------------+-----+--------+-------------------+--------+--------------------+--------------------+---------+



24/05/25 21:01:20 WARN DAGScheduler: Broadcasting large task binary with size 1418.6 KiB


+--------+--------------------+--------------------+-------------+-----+--------+-------------------+--------+--------------------+--------------------+---------+-----------+--------------------+--------------------+--------------------+--------------------+------------+------------+--------------------+-----+--------------------+--------------------+----------+--------------------+
|     aid|               title|                 url|       domain|votes|    user|          posted_at|comments|        source_title|         source_text|frontpage|text_length|               words|      filtered_words|         tf_features|      tfidf_features|domain_index|  domain_vec|            features|label|       rawPrediction|         probability|prediction|corrected_prediction|
+--------+--------------------+--------------------+-------------+-----+--------+-------------------+--------+--------------------+--------------------+---------+-----------+--------------------+--------------------+------------

24/05/25 21:01:21 WARN DAGScheduler: Broadcasting large task binary with size 1414.7 KiB


In [20]:
ssc_t.stop()

----- Stopping... this may take a few seconds -----


24/05/25 21:04:13 ERROR ReceiverTracker: Deregistered receiver for stream 0: Stopped by driver
24/05/25 21:04:13 WARN SocketReceiver: Error receiving data
java.net.SocketException: Socket closed
	at java.net.SocketInputStream.socketRead0(Native Method)
	at java.net.SocketInputStream.socketRead(SocketInputStream.java:116)
	at java.net.SocketInputStream.read(SocketInputStream.java:171)
	at java.net.SocketInputStream.read(SocketInputStream.java:141)
	at sun.nio.cs.StreamDecoder.readBytes(StreamDecoder.java:284)
	at sun.nio.cs.StreamDecoder.implRead(StreamDecoder.java:326)
	at sun.nio.cs.StreamDecoder.read(StreamDecoder.java:178)
	at java.io.InputStreamReader.read(InputStreamReader.java:184)
	at java.io.BufferedReader.fill(BufferedReader.java:161)
	at java.io.BufferedReader.readLine(BufferedReader.java:324)
	at java.io.BufferedReader.readLine(BufferedReader.java:389)
	at org.apache.spark.streaming.dstream.SocketReceiver$$anon$2.getNext(SocketInputDStream.scala:121)
	at org.apache.spark.str

In [28]:
# Load saved predictions
predictions_df = spark.read.option("header", "true").csv("/Users/xiaodi/anaconda3/spark/notebooks/predictions")

# Define a functions to convert numbers to boolen
def bool_to_double(value):
    return 1.0 if value == "true" else 0.0 if value == "false" else None

bool_to_double_udf = udf(bool_to_double, DoubleType())
predictions_df = predictions_df.withColumn("frontpage_num", bool_to_double_udf(predictions_df["frontpage"]))

# Check for the lines
predictions_df.show(n=5)

# Compute the correct predictions and accuracy
correct_predictions = predictions_df.filter(predictions_df["frontpage_num"] == predictions_df["corrected_prediction"]).count()
total_predictions = predictions_df.count()
true_case = predictions_df.filter(predictions_df["frontpage_num"] == 1.0).count()
false_case = predictions_df.filter(predictions_df["frontpage_num"] == 0.0).count()

accuracy = correct_predictions / total_predictions

# Calculate other evaluation metrics
tp = predictions_df.filter((predictions_df["frontpage_num"] == 1.0) & (predictions_df["corrected_prediction"] == 1.0)).count()  # True Positives
tn = predictions_df.filter((predictions_df["frontpage_num"] == 0.0) & (predictions_df["corrected_prediction"] == 1.0)).count()  # True Negatives
fp = predictions_df.filter((predictions_df["frontpage_num"] == 0.0) & (predictions_df["corrected_prediction"] == 1.0)).count()  # False Positives
fn = predictions_df.filter((predictions_df["frontpage_num"] == 1.0) & (predictions_df["corrected_prediction"] == 0.0)).count()  # False Negatives

precision = tp / (tp + fp) if tp + fp != 0 else 0  # Precision
recall = tp / (tp + fn) if tp + fn != 0 else 0  # Recall
f1 = 2 * precision * recall / (precision + recall) if precision + recall != 0 else 0  # F1 Score

print(f"Correct Predictions: {correct_predictions}")
print(f"Total Predictions: {total_predictions}")
print("Frontpage = True:", true_case)
print("Frontpage = False:", false_case)
print(f"Accuracy: {accuracy}")
print("Precision: ", precision)
print("Recall: ", recall)
print("F1 Score: ", f1)


+--------+--------------------+--------------------+---------------+-----+------------+-------------------+--------+---------+--------------------+-------------+
|     aid|               title|                 url|         domain|votes|        user|          posted_at|comments|frontpage|corrected_prediction|frontpage_num|
+--------+--------------------+--------------------+---------------+-----+------------+-------------------+--------+---------+--------------------+-------------+
|40474069|Triathlon Forum: ...|https://forum.slo...| slowtwitch.com|    1|        wslh|2024-05-25 10:32:16|       0|    false|                 0.0|          0.0|
|40473550|Big tech has dist...|https://www.thegu...|theguardian.com|    3|    lastdong|2024-05-25 08:28:56|       0|     true|                 0.0|          1.0|
|40474234|NASA finds more i...|https://arstechni...|arstechnica.com|    1| andreiursan|2024-05-25 11:25:38|       0|    false|                 0.0|          0.0|
|40474064|If You Wish to Un.